In [10]:
# Basic models
import traceback
import numpy as np
import pandas as pd

# Data operation models
from sklearn.model_selection import StratifiedKFold
from sklearn import cross_validation
from sklearn.model_selection import ParameterGrid

# Classifier models
from sklearn.svm import SVC

# Evaluation models
import sklearn.metrics as sm

In [11]:
# Configurations for the model
def run_Config():
    data_name = 'Cardiac_Death_dummy.xlsx'
    k_holdout = 10
    k_cv = 5
    origin_pars = {
            'indicator_name': ['macc'],
            'kernel': ['rbf'],
            'C': [0.1,1,10,100],
            'gamma': [2,3],
            'class_weight':[{1:0.4,2:0.6},{1:0.5,2:0.5}]
            }
    pars = list(ParameterGrid(origin_pars))
    return data_name, k_holdout, k_cv, pars

In [12]:
# Load data
def run_load_data(data_name):
    all = pd.read_excel(data_name)
    all = all.values
    all_fea = all[:,:-1]
    all_label = get_normal_label(all[:,-1])
    return all_fea, all_label

def get_normal_label(y):
    y_uni = np.unique(np.array(y))
    for i in xrange(len(y_uni)):
        y[np.nonzero(y == y_uni[i])[0]] = i+1
    return y

In [13]:
# Run both HoldoutCV and GridSearchCV
def run_doubleCV(all_fea, all_label, k_holdout, k_cv, pars):
    # HoldoutCV
    i_t = 0
    res_list = []
    opt_pars_list = []
    holdoutcv = StratifiedKFold(n_splits = k_holdout, shuffle = True)
    for i_learn, i_test in holdoutcv.split(all_fea, all_label):
        # Times
        i_t = i_t + 1
        print 'Round ', str(i_t), ' Holdout CV----------------------'
        
        # Obtain current learning and heldout data
        learn_fea, test_fea = all_fea[i_learn], all_fea[i_test]
        learn_label, test_label = all_label[i_learn], all_label[i_test]
        
        # GridSearchCV
        j_t = 0
        optdata = {'score':0}
        for i_pars in pars:
            # times
            j_t = j_t + 1
            print 'round ', str(j_t), ' gridsearch cv----------------'
            pars_score = []
            gridcv = StratifiedKFold(n_splits = k_cv, shuffle = True)
            for i_train, i_valid in gridcv.split(learn_fea, learn_label):
                # obtain current training and validation data
                train_fea, valid_fea = learn_fea[i_train], learn_fea[i_valid]
                train_label, valid_label = learn_label[i_train], learn_label[i_valid]
                # learn the model
                # i_pars = {'par_name1':par1,'par_name2':par2,...,'par_nameN':parN}
                valid_pre = run_model(i_pars, train_fea, train_label, valid_fea)
                grid_score = run_validation(valid_pre, valid_label, i_pars['indicator_name'])
                pars_score.append(grid_score)
            if np.mean(pars_score) > optdata['score']:
                optdata['pars'] = i_pars
                optdata['score'] = np.mean(pars_score)
                    
        # Holdout testing
        # best_pars is a dict too
        best_pars = optdata['pars']
        test_pre = run_model(best_pars, learn_fea, learn_label, test_fea)
        
        # Evaluate the prediction
        res_now = run_evaluation(test_pre, test_label)
        
        # Save results
        res_list.append(res_now)
        opt_pars_list.append(best_pars)
        
    return res_list, opt_pars_list

In [14]:
# Fit the model and make the prediction
def run_model(p, X, y, Z):
    clf = SVC(C=p['C'], kernel=p['kernel'], gamma=p['gamma'], class_weight=p['class_weight'])
    clf.fit(X,y)
    zpre = clf.predict(Z)
    print 'Model is running...'
    return zpre

In [15]:
# Obtain the score
def run_validation(zpre, z, s_name):
    i_pos = np.nonzero(z == max(z))
    i_neg = np.nonzero(z == min(z))
    tpr = (1 - sm.hamming_loss(z[i_pos], zpre[i_pos]))*100
    tnr = (1 - sm.hamming_loss(z[i_neg], zpre[i_neg]))*100
    if s_name.lower() == 'macc':
        s = 0.5*(tpr+tnr)
    elif s_name.lower() == 'gm':
        s = np.sqrt(tpr*tnr)
    elif s_name.lower() == 'tpr':
        s = tpr
    else: # error
        s = sum((1 if i_pre == i_true else 0 for i_pre, i_true in zip(zpre,z)))/float(len(z))
    return s

In [16]:
# Evaluate the performance
def run_evaluation(p, y):
    res_dict = {}
    i_pos = np.nonzero(y == max(y))
    i_neg = np.nonzero(y == min(y))
    res_dict['TPR'] = (1 - sm.hamming_loss(y[i_pos], p[i_pos]))*100
    res_dict['TNR'] = (1 - sm.hamming_loss(y[i_neg], p[i_neg]))*100
    res_dict['MAcc'] = np.mean([res_dict['TPR'], res_dict['TNR']])
    res_dict['GM'] = np.sqrt(res_dict['TPR']*res_dict['TNR'])
    res_dict['F1(Macro)'] = sm.f1_score(y, p, average='macro')*100
    res_dict['F1(Micro)'] = sm.f1_score(y, p, average='micro')*100
    res_dict['Acc'] = sm.accuracy_score(y, p)*100
    return res_dict

In [17]:
def get_dict(d):
    list_final = []
    for i in d:
        list_now = i + '_' + str(d[i])
        list_final.append(list_now)
    return list_final

In [18]:
if __name__ == '__main__':
    data_name, k_holdout, k_cv, pars = run_Config()
    all_fea, all_label = run_load_data(data_name)
    res_list, opt_pars_list = run_doubleCV(all_fea, all_label, k_holdout, k_cv, pars)
    print 'Each heldout cv result:'
    print '-----------------------'
    for i, j in zip(res_list,opt_pars_list):
        print get_dict(i), 'with hyper-parameters:'
        print get_dict(j)
        print '-----------------------'

Round  1  Holdout CV----------------------
round  1  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  2  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  3  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  4  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  5  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  6  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  7  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running.

//anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  7  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  8  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  9  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  10  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  11  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  12  gridsearch cv----------------
Model is running...
Model is running...
Model is running...
Model is running...
Model is running...
round  13  gridsearch cv----------------
Model is running

KeyboardInterrupt: 